In [2]:
import pyqir
from pyqir import (
    BasicBlock,
    Builder,
    Context,
    Function,
    Linkage,
    Module,
    ModuleFlagBehavior,
)

In [3]:
context = Context()
mod = pyqir.qir_module(
    context,
    "dynamic_allocation",
    qir_major_version=1,
    qir_minor_version=0,
    dynamic_qubit_management=True,
    dynamic_result_management=True,
)
builder = Builder(context)

# define external calls and type definitions
qubit_type = pyqir.qubit_type(context)
result_type = pyqir.result_type(context)
int_type = pyqir.IntType(context, 32)
double_type = pyqir.Type.double(context)

class CustomArray(pyqir.ArrayType):
    def __int__(self):
        print("hello")

# twod_array_type = pyqir.ArrayType(array_type)
ex_double = pyqir.const(double_type, 5.0)

# PyQIR assumes you want to use static allocation for qubits and results, but
# you can still use dynamic allocation by manually calling the appropriate
# runtime functions.
qubit_allocate = Function(
    pyqir.FunctionType(qubit_type, []),
    Linkage.EXTERNAL,
    "__quantum__rt__qubit_allocate",
    mod,
)

qubit_release = Function(
    pyqir.FunctionType(pyqir.Type.void(context), [qubit_type]),
    Linkage.EXTERNAL,
    "__quantum__rt__qubit_release",
    mod,
)

result_get_one = Function(
    pyqir.FunctionType(result_type, []),
    Linkage.EXTERNAL,
    "__quantum__rt__result_get_one",
    mod,
)

result_equal = Function(
    pyqir.FunctionType(pyqir.IntType(context, 1), [result_type, result_type]),
    Linkage.EXTERNAL,
    "__quantum__rt__result_equal",
    mod,
)

m = Function(
    pyqir.FunctionType(result_type, [qubit_type]),
    Linkage.EXTERNAL,
    "__quantum__qis__m__body",
    mod,
)

ex_f = Function(
    pyqir.FunctionType(pyqir.Type.void(context), [qubit_type, double_type]),
    Linkage.INTERNAL,
    "ex_f",
    mod
)



# Create entry point
num_qubits = 1
num_results = 1
entry_point = pyqir.entry_point(mod, "main", num_qubits, num_results)
pyqir.add_string_attribute(ex_f, "qub", "val_qub")
builder.insert_at_end(BasicBlock(context, "entry", entry_point))

# Define entry point body
qubit_return = builder.call(qubit_allocate, [])

assert qubit_return is not None
qubit = qubit_return

qis = pyqir.BasicQisBuilder(builder)
qis.h(qubit)

# Instead of qis.mz, use __quantum__qis__m__body.
result = builder.call(m, [qubit])
assert result is not None

# Instead of if_result, use __quantum__rt__result_equal and mod.if_.
one = builder.call(result_get_one, [])
assert one is not None
result_is_one = builder.call(result_equal, [result, one])
assert result_is_one is not None
builder.if_(result_is_one, lambda: qis.reset(qubit))
builder.call(ex_f, [qubit, ex_double])
block_ex_f = BasicBlock(context, "block1", ex_f)
builder.insert_at_end(block_ex_f)
builder.add(ex_double, ex_double)
qubit_return = builder.call(qubit_allocate, [])
pyqir.BasicQisBuilder(builder).x(qubit)
pyqir.BasicQisBuilder(builder).rx(ex_f.params[1], qubit)

# Be sure to release any allocated qubits when you're done with them.
builder.call(qubit_release, [qubit])

TypeError: type 'builtins.ArrayType' is not an acceptable base type

In [35]:
print(str(mod))

; ModuleID = 'dynamic_allocation'
source_filename = "dynamic_allocation"

%Qubit = type opaque
%Result = type opaque

declare %Qubit* @__quantum__rt__qubit_allocate()

declare void @__quantum__rt__qubit_release(%Qubit*)

declare %Result* @__quantum__rt__result_get_one()

declare i1 @__quantum__rt__result_equal(%Result*, %Result*)

declare %Result* @__quantum__qis__m__body(%Qubit*)

define internal void @ex_f(%Qubit* %0, double %1) #0 {
block1:
  %2 = call %Qubit* @__quantum__rt__qubit_allocate()
  call void @__quantum__qis__x__body(%Qubit* %0)
  call void @__quantum__qis__rx__body(double %1, %Qubit* %0)
  call void @__quantum__rt__qubit_release(%Qubit* %0)
}

define void @main() #1 {
entry:
  %0 = call %Qubit* @__quantum__rt__qubit_allocate()
  call void @__quantum__qis__h__body(%Qubit* %0)
  %1 = call %Result* @__quantum__qis__m__body(%Qubit* %0)
  %2 = call %Result* @__quantum__rt__result_get_one()
  %3 = call i1 @__quantum__rt__result_equal(%Result* %1, %Result* %2)
  br i1 %3, labe